In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.cluster import KMeans

In [2]:
customers=pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/olist_customers_dataset.csv")
orders=pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/olist_orders_dataset.csv",parse_dates=['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',
                         'order_delivered_customer_date',
                         'order_estimated_delivery_date'])
order_items=pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/olist_order_items_dataset.csv",parse_dates=['shipping_limit_date'])
order_payments= pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/olist_order_payments_dataset.csv")
products=pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/olist_products_dataset.csv", usecols=['product_id','product_category_name'])
product_name_translation=pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/product_category_name_translation.csv")
order_reviews= pd.read_csv("https://raw.githubusercontent.com/JyothirmaiV13/Quest-labs-Ai/main/olist_order_reviews_dataset.csv", usecols=['review_score','order_id'])

In [3]:
products['product_category_name']=products['product_category_name'].replace(product_name_translation.set_index('product_category_name')['product_category_name_english'].to_dict())

In [4]:
products.head()

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares


In [5]:
df1 = order_items.merge(order_payments, on='order_id')
df2 = df1.merge(orders, on='order_id')
df3 = df2.merge(customers, on='customer_id')
df4=df3.merge(order_reviews, on='order_id')
df=df4.merge(products, on='product_id')

In [6]:
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_score,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1,credit_card,2,...,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,5,cool_stuff
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1,boleto,1,...,2017-06-29 02:44:11,2017-07-05 12:00:33,2017-07-13 20:39:29,2017-07-26,0fb8e3eab2d3e79d92bb3fffbb97f188,75800,jatai,GO,5,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1,credit_card,2,...,2018-05-18 12:31:43,2018-05-23 14:05:00,2018-06-04 18:34:26,2018-06-07,3419052c8c6b45daf79c1e426f9e9bcb,30720,belo horizonte,MG,4,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1,credit_card,3,...,2017-08-01 18:55:08,2017-08-02 19:07:36,2017-08-09 21:26:33,2017-08-25,e7c828d22c0682c1565252deefbe334d,83070,sao jose dos pinhais,PR,5,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1,credit_card,4,...,2017-08-10 22:05:11,2017-08-11 19:43:07,2017-08-24 20:04:21,2017-09-01,0bb98ba72dcc08e95f9d8cc434e9a2cc,36400,conselheiro lafaiete,MG,5,cool_stuff


In [7]:
df.isnull().sum()

order_id                            0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date     1235
order_delivered_customer_date    2471
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
review_score                        0
product_category_name            1695
dtype: int64

In [8]:
df[df['order_delivered_customer_date'].isnull()]['order_status'].value_counts()

order_status
shipped        1167
canceled        546
invoiced        370
processing      370
delivered         8
unavailable       7
approved          3
Name: count, dtype: int64

In [9]:
df=df[df['order_status']=='delivered']
df = df.dropna(subset=['product_category_name'])


In [10]:
df.isnull().sum()

order_id                          0
order_item_id                     0
product_id                        0
seller_id                         0
shipping_limit_date               0
price                             0
freight_value                     0
payment_sequential                0
payment_type                      0
payment_installments              0
payment_value                     0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
customer_unique_id                0
customer_zip_code_prefix          0
customer_city                     0
customer_state                    0
review_score                      0
product_category_name             0
dtype: int64

Calculating average order value, number of purchases based on the category, Frequently brought together items, and Average rating

In [11]:
from itertools import combinations
from collections import Counter

df_new=df.copy()
df_new['order_value']=df.groupby('order_id')['price'].transform('sum')

average_order_value=df_new.groupby('customer_id')['order_value'].mean().reset_index()
average_order_value.columns=['customer_id','average_order_value']

purchases_per_category=df_new.groupby(['customer_id','product_category_name']).size().reset_index(name='number_of_purchases')

order_products=df_new.groupby('order_id')['product_id'].apply(list)
combo_counts=Counter()
for products in order_products:
  combos=combinations(products,2)
  combo_counts.update(combos)
frequently_bought_together=pd.DataFrame(combo_counts.items(),columns=['product_pair','frequency']).sort_values(by='frequency',ascending=False)


df_ratings=df[['customer_id','product_id','review_score']]
average_rating=df_ratings.groupby('product_id')['review_score'].mean().reset_index()
average_rating.columns=['product_id','average_rating']


#Displaying the final results
print("Average Order Value for Each User:")
print(average_order_value)
print("\nNumber of Purchases in a Category for Each User:")
print(purchases_per_category)
print("\nFrequently Bought Together Items:")
print(frequently_bought_together)
print("\nAverage Rating for Each Item:")
print(average_rating)

Average Order Value for Each User:
                            customer_id  average_order_value
0      00012a2ce6f8dcda20d059ce98491703                89.80
1      000161a058600d5901f007fab4c27140                54.90
2      0001fd6190edaaf884bcaf3d49edf079               179.99
3      0002414f95344307404f0ace7a26f1d5               149.90
4      000379cdec625522490c315e70c7a9fb                93.00
...                                 ...                  ...
94502  fffcb937e9dd47a13f05ecb8290f4d3e                78.00
94503  fffecc9f79fd8c764f843e9951b11341               164.70
94504  fffeda5b6d849fbd39689bb92087f431                47.90
94505  ffff42319e9b2d713724ae527742af25               199.90
94506  ffffa3172527f765de70084a7e53aae8                21.80

[94507 rows x 2 columns]

Number of Purchases in a Category for Each User:
                            customer_id   product_category_name  \
0      00012a2ce6f8dcda20d059ce98491703                    toys   
1      000161a058600d59

In [12]:
prods=df['product_id']
ratng=df['review_score']

n_prods=len(prods)
n_ratgs=len(ratng)
print(prods.count(),ratng.count())
df_final=df

113233 113233


In [14]:
!pip install torch torchvision torchaudio
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_prods, n_ratgs, n_factors=20):
        super().__init__()
        # create product embeddings
        self.product_factors = torch.nn.Embedding(n_prods, n_factors) # think of this as a lookup table for the input.
        # create rating embeddings
        self.rating_factors = torch.nn.Embedding(n_ratgs, n_factors) # think of this as a lookup table for the input.
        self.product_factors.weight.data.uniform_(0, 0.05)
        self.rating_factors.weight.data.uniform_(0, 0.05)
 # matrix multiplication
    def forward(self, data):
      products, ratings = data[:,0], data[:,1]
      return (self.user_factors(products)*self.item_factors(ratings)).sum(1)


    def predict(self, product, rating):
      return self.forward(product, rating)

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [16]:
df_final=df_final.drop_duplicates(subset='product_id')
df_final=df_final[:1000]
#Creating the interaction matrix of products and users based on ratings and replacing NaN value with 0
final_ratings_matrix = df_final.pivot(index = 'product_id', columns ='customer_id', values = 'review_score').fillna(0)
print('Shape of final_ratings_matrix: ', final_ratings_matrix.shape)

#Finding the number of non-zero entries in the interaction matrix
given_num_of_ratings = np.count_nonzero(final_ratings_matrix)
print('given_num_of_ratings = ', given_num_of_ratings)

#Finding the possible number of ratings as per the number of users and products
possible_num_of_ratings = final_ratings_matrix.shape[0] * final_ratings_matrix.shape[1]
print('possible_num_of_ratings = ', possible_num_of_ratings)

#Density of ratings
density = (given_num_of_ratings/possible_num_of_ratings)
density *= 100
print ('density: {:4.2f}%'.format(density))

final_ratings_matrix.head()

Shape of final_ratings_matrix:  (1000, 965)
given_num_of_ratings =  1000
possible_num_of_ratings =  965000
density: 0.10%


customer_id,004a2b3e8b026284407c620f4f01f686,010583f85dd55d32a6db1068f58ca6f1,01952039ba4abb6c459e7b6d62a69d7a,0241bf6728f2c70f948e9a51484d1435,02cb822eec8b6309e7ce715ccea3390d,02d2c49ef9837fbf18d71940dd7cf97f,02e460a9db12a0de7f952f8facb66312,034c3cb26461c58520139e4efb33ed90,03a7750fc7a7bfbd7a84b2f4f26b92f1,03d804d4d3e76cbac2fdec148e8d2742,...,fd8a9ce7e554b0a3b083521837d0f7c0,fd967f807427c24072652b1f62d4fe47,fe2b285a4d3936b9a37ff7460a0d403f,fe43b2f349c9ab46883ce0e02491432c,fe9cb076acaffbee858187352e2f7266,ff60d321ad354e757bb24fa7ae799676,ff63f83bb3b2dc9a2df626cefbddda8b,ff70b0c5553479bb9130858713a7e05b,ffca23943c4b620ea24812018373fa3f,fff5169e583fd07fac9fec88962f189d
product_id,,,,,,,,,,,,,,,,,,,,,
009af1277432f1a05742df69cdf72d1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ffe57f0110d73fd84d162252b2c784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
013ee64977aaa6b2b25475095162e0e9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
016711f78a6a87696645db0a6a834826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
017692475c1c954ff597feda05131d73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.neighbors import NearestNeighbors

matrix=final_ratings_matrix.values
knn= NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(matrix)

# Function to recommend products for a given product_id
def recommend_products(product_id, final_ratings_matrix, n_recommendations=5):
    product_index = final_ratings_matrix.index.get_loc(product_id)
    distances, indices = knn.kneighbors(matrix[product_index].reshape(1, -1), n_neighbors=n_recommendations + 1)
    recommendations = []

    for i in range(1, len(distances[0])):
        recommendations.append(final_ratings_matrix.index[indices[0][i]])

    return recommendations


product_id_to_recommend = '00ffe57f0110d73fd84d162252b2c784'
recommended_products = recommend_products(product_id_to_recommend, final_ratings_matrix)
print(f"Products recommended for product_id {product_id_to_recommend}: {recommended_products}")

Products recommended for product_id 00ffe57f0110d73fd84d162252b2c784: ['aacfae7cd4bac4849766f640abf2db8a', 'aa8d88eb4b9cb38894e33fa624c4287f', 'aa8627a375771ab01288705307ec4ae5', 'ab1f9387c0627dd24000bfbc54fdeee0', 'aa280035c50ba62c746480a59045eec4']


Ranking model to score each candidate product based on its predicted relevance to the user

In [18]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357245 sha256=568b53ad005df115a824b7c9dc4e3a09fd010d78cf31f1bd3eeea146eb1de03a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [21]:
print(df_final.dtypes)

order_id                                 object
order_item_id                             int64
product_id                               object
seller_id                                object
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
payment_sequential                        int64
payment_type                             object
payment_installments                      int64
payment_value                           float64
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
customer_unique_id                       object
customer_zip_code_prefix                  int64
customer_city                           

In [30]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split


df=df_final.copy()

# Load additional product information
product_info=df_final[['product_category_name','product_id']].copy()
#product_info[['product_category_name','product_id']]= df_final[['product_category_name','product_id']]
#product_info=product_info.assign(product_category_name=df_final['product_category_name'],product_id=df_final['product_id'])
# Merge the pivot DataFrame with product information
pivot_with_info = pd.merge(df, product_info, on='product_id')

# Convert the DataFrame to the required format for the Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['customer_id', 'product_id', 'review_score']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train an SVD model
svd = SVD()
svd.fit(trainset)

# Function to recommend products for a given user
def recommend_products(customer_id, df, svd, product_info, n_recommendations=5):
    # Get the list of all product_ids
    product_ids = df['product_id'].unique()

    # Predict the relevance score for each product for the given user
    scores = []
    for product_id in product_ids:
        predicted_score = svd.predict(customer_id, product_id).est
        scores.append((product_id, predicted_score))

    # Sort the products based on the predicted scores in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    # Get the top n recommendations
    top_scores = scores[:n_recommendations]

    # Get product details
    recommended_products = product_info[product_info['product_id'].isin([score[0] for score in top_scores])]

    # Add predicted scores to the recommendations
    recommended_products['predicted_score'] = [score[1] for score in top_scores]

    return recommended_products


customer_id_to_recommend = '004a2b3e8b026284407c620f4f01f686'
recommended_products = recommend_products(customer_id_to_recommend, df, svd, product_info)

# Display the recommendations in a user-friendly format
def display_recommendations(recommendations):
    print("Top Recommended Products:\n")
    for index, row in recommendations.iterrows():
        print(f"Product Category Name: {row['product_category_name']}")
        print(f"Predicted Score: {row['predicted_score']:.2f}")
        print("-" * 40)

# Display the recommendations
display_recommendations(recommended_products)



Top Recommended Products:

Product Category Name: sports_leisure
Predicted Score: 4.55
----------------------------------------
Product Category Name: musical_instruments
Predicted Score: 4.47
----------------------------------------
Product Category Name: housewares
Predicted Score: 4.47
----------------------------------------
Product Category Name: toys
Predicted Score: 4.44
----------------------------------------
Product Category Name: stationery
Predicted Score: 4.43
----------------------------------------


<ipython-input-30-440c62547b1c>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_products['predicted_score'] = [score[1] for score in top_scores]
